In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

class HypothesisTester:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        self.results = {}
        
    def calculate_risk_metrics(self):
        """Calculate risk metrics: Claim Frequency and Claim Severity"""
        self.data['has_claim'] = self.data['TotalClaims'] > 0
        self.data['claim_severity'] = self.data.apply(
            lambda x: x['TotalClaims'] if x['has_claim'] else np.nan, axis=1
        )
        
    def test_province_risk(self):
        """Test H₀: There are no risk differences across provinces"""
        # Claim Frequency test
        province_groups = self.data.groupby('Province')
        claim_counts = province_groups['has_claim'].sum()
        total_counts = province_groups.size()
        
        # Chi-squared test for proportions
        contingency_table = pd.crosstab(self.data['Province'], self.data['has_claim'])
        chi2, p_freq, dof, expected = stats.chi2_contingency(contingency_table)
        
        # Claim Severity test (ANOVA)
        severity_data = [group['claim_severity'].dropna() 
                        for name, group in province_groups]
        f_stat, p_sev = stats.f_oneway(*severity_data)
        
        self.results['province_risk'] = {
            'claim_frequency': {'chi2': chi2, 'p_value': p_freq},
            'claim_severity': {'f_stat': f_stat, 'p_value': p_sev}
        }
        
        return p_freq < 0.05 or p_sev < 0.05
    
    def test_zipcode_risk(self):
        """Test H₀: There are no risk differences between zip codes"""
        # Sample top 10 zip codes for testing (practical approach)
        top_zips = self.data['ZipCode'].value_counts().head(10).index
        sample_data = self.data[self.data['ZipCode'].isin(top_zips)]
        
        # ANOVA for claim severity
        zip_groups = [group['claim_severity'].dropna() 
                     for name, group in sample_data.groupby('ZipCode')]
        f_stat, p_sev = stats.f_oneway(*zip_groups)
        
        # Chi-squared for claim frequency
        contingency_table = pd.crosstab(sample_data['ZipCode'], 
                                       sample_data['has_claim'])
        chi2, p_freq, dof, expected = stats.chi2_contingency(contingency_table)
        
        self.results['zipcode_risk'] = {
            'claim_frequency': {'chi2': chi2, 'p_value': p_freq},
            'claim_severity': {'f_stat': f_stat, 'p_value': p_sev}
        }
        
        return p_freq < 0.05 or p_sev < 0.05
    
    def test_zipcode_margin(self):
        """Test H₀: There is no significant margin difference between zip codes"""
        self.data['margin'] = self.data['TotalPremium'] - self.data['TotalClaims']
        
        # Sample top 10 zip codes
        top_zips = self.data['ZipCode'].value_counts().head(10).index
        sample_data = self.data[self.data['ZipCode'].isin(top_zips)]
        
        # One-way ANOVA
        zip_groups = [group['margin'].dropna() 
                     for name, group in sample_data.groupby('ZipCode')]
        f_stat, p_val = stats.f_oneway(*zip_groups)
        
        # Post-hoc Tukey test if significant
        if p_val < 0.05:
            tukey = pairwise_tukeyhsd(
                endog=sample_data['margin'].dropna(),
                groups=sample_data['ZipCode'],
                alpha=0.05
            )
            self.results['zipcode_margin_posthoc'] = tukey.summary()
        
        self.results['zipcode_margin'] = {'f_stat': f_stat, 'p_value': p_val}
        return p_val < 0.05
    
    def test_gender_risk(self):
        """Test H₀: There is no significant risk difference between Women and Men"""
        # Filter for gender data
        gender_data = self.data[self.data['Gender'].isin(['Male', 'Female'])]
        
        # Claim Frequency test (two-proportion z-test)
        male_claims = gender_data[gender_data['Gender'] == 'Male']['has_claim'].sum()
        male_total = (gender_data['Gender'] == 'Male').sum()
        female_claims = gender_data[gender_data['Gender'] == 'Female']['has_claim'].sum()
        female_total = (gender_data['Gender'] == 'Female').sum()
        
        count = np.array([male_claims, female_claims])
        nobs = np.array([male_total, female_total])
        z_stat, p_freq = proportions_ztest(count, nobs)
        
        # Claim Severity test (t-test)
        male_severity = gender_data[gender_data['Gender'] == 'Male']['claim_severity'].dropna()
        female_severity = gender_data[gender_data['Gender'] == 'Female']['claim_severity'].dropna()
        
        t_stat, p_sev = stats.ttest_ind(male_severity, female_severity, 
                                       equal_var=False, nan_policy='omit')
        
        self.results['gender_risk'] = {
            'claim_frequency': {'z_stat': z_stat, 'p_value': p_freq},
            'claim_severity': {'t_stat': t_stat, 'p_value': p_sev}
        }
        
        return p_freq < 0.05 or p_sev < 0.05
    
    def generate_report(self):
        """Generate comprehensive report"""
        report = "# Hypothesis Testing Results\n\n"
        
        for test_name, results in self.results.items():
            report += f"## {test_name.replace('_', ' ').title()}\n"
            
            if 'claim_frequency' in results:
                freq = results['claim_frequency']
                sev = results['claim_severity']
                
                report += f"### Claim Frequency\n"
                report += f"- Test Statistic: {freq.get('chi2', freq.get('z_stat', 'N/A')):.4f}\n"
                report += f"- P-value: {freq['p_value']:.6f}\n"
                report += f"- Significant: {'YES' if freq['p_value'] < 0.05 else 'NO'}\n\n"
                
                report += f"### Claim Severity\n"
                report += f"- Test Statistic: {sev.get('f_stat', sev.get('t_stat', 'N/A')):.4f}\n"
                report += f"- P-value: {sev['p_value']:.6f}\n"
                report += f"- Significant: {'YES' if sev['p_value'] < 0.05 else 'NO'}\n\n"
            
            else:
                stats_data = results
                report += f"- Test Statistic: {stats_data.get('f_stat', stats_data.get('t_stat', 'N/A')):.4f}\n"
                report += f"- P-value: {stats_data['p_value']:.6f}\n"
                report += f"- Significant: {'YES' if stats_data['p_value'] < 0.05 else 'NO'}\n\n"
            
            # Business interpretation
            if test_name == 'province_risk' and (freq['p_value'] < 0.05 or sev['p_value'] < 0.05):
                report += "**Business Implication:** Regional risk differences detected. Consider adjusting premiums by province or implementing region-specific underwriting rules.\n\n"
            
            elif test_name == 'gender_risk' and (freq['p_value'] < 0.05 or sev['p_value'] < 0.05):
                report += "**Business Implication:** Gender-based risk differences found. While pricing by gender may be restricted in some regions, this information can inform marketing and product design.\n\n"
        
        return report